<a href="https://colab.research.google.com/github/Praveencyber08/infosys/blob/main/Sp_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# ============================================================
# Modern SaaS Dashboard + Forecasting (Polished, aligned output)
# Single cell — Jupyter / Colab
# Logo path (local): /mnt/data/734655bf-327e-4fd5-bcbc-054bb619f15d.png
# ============================================================

# NOTE: If running in fresh Colab, uncomment installs:
# !pip install yfinance textblob feedparser ipywidgets seaborn matplotlib pillow statsmodels scikit-learn tensorflow --quiet
# !python -m textblob.download_corpora

import os, textwrap, math, warnings
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.table as mtable
import seaborn as sns
from textblob import TextBlob
from PIL import Image
from IPython.display import display, clear_output

warnings.filterwarnings("ignore")

# optional libs (graceful fallback)
try:
    import yfinance as yf
except Exception:
    yf = None

try:
    import feedparser
except Exception:
    feedparser = None

try:
    import ipywidgets as widgets
except Exception:
    widgets = None

# forecasting libs (optional)
try:
    from statsmodels.tsa.arima.model import ARIMA
    STATSMODELS_AVAILABLE = True
except Exception:
    STATSMODELS_AVAILABLE = False

try:
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout
    from sklearn.preprocessing import MinMaxScaler
    TENSORFLOW_AVAILABLE = True
except Exception:
    TENSORFLOW_AVAILABLE = False

# ---------------- Configuration ----------------
LOGO_PATH = "/mnt/data/734655bf-327e-4fd5-bcbc-054bb619f15d.png"
EXPORT_DIR = "data"
os.makedirs(EXPORT_DIR, exist_ok=True)

COMPANIES = [
    "Apple","Google","Amazon","Microsoft","Meta",
    "Tesla","Netflix","Intel","IBM","NVIDIA"
]

SYMBOLS = {
    "Apple":"AAPL","Google":"GOOGL","Amazon":"AMZN","Microsoft":"MSFT",
    "Meta":"META","Tesla":"TSLA","Netflix":"NFLX","Intel":"INTC",
    "IBM":"IBM","NVIDIA":"NVDA"
}

# Visual theme
BG      = "#f5f7fa"
CARD_BG = "#ffffff"
ACCENT  = "#2b7cff"
MUTED   = "#6b7280"
HEADER  = "#0f1724"
POS     = "#16a34a"
NEG     = "#ef4444"
TABLE_HEADER_BG = "#f1f5f9"

# ---------------- Helpers ----------------
def safe_rss_titles(q, max_items=5):
    if feedparser is None:
        return []
    try:
        from urllib.parse import quote
        url = f"https://news.google.com/rss/search?q={quote(q)}&hl=en-US&gl=US&ceid=US:en"
        feed = feedparser.parse(url)
        return [e.get("title","") for e in feed.entries[:max_items]]
    except Exception:
        return []

def tb_sentiment(txt):
    if not txt or not isinstance(txt, str):
        return "Neutral", 0.0
    p = TextBlob(txt).sentiment.polarity
    if p > 0.05: return "Positive", round(float(p), 3)
    if p < -0.05: return "Negative", round(float(p), 3)
    return "Neutral", 0.0

def simulate_trends(keys, days=90, seed=42):
    np.random.seed(seed)
    dates = pd.date_range(end=datetime.today(), periods=days)
    df = pd.DataFrame(index=dates)
    for k in keys:
        base = np.abs(np.random.normal(30, 10, days))
        for _ in range(np.random.randint(0, 3)):
            c = np.random.randint(5, days-5); w = np.random.randint(2,6)
            base += np.exp(-0.5 * ((np.arange(days)-c)/w)**2) * np.random.randint(6,80)
        df[k] = (base / base.max() * 100).round(1)
    return df

def fetch_stock_close(symbol, days=720):
    if yf is None or not symbol:
        return pd.DataFrame()
    try:
        df = yf.download(symbol, period=f"{days}d", progress=False)
        if df.empty: return pd.DataFrame()
        df2 = df.reset_index()[["Date","Close","Volume"]].rename(columns={"Date":"date","Close":"close"})
        df2["date"] = pd.to_datetime(df2["date"])
        return df2.sort_values("date")
    except Exception:
        return pd.DataFrame()

def safe_latest_price(df):
    if isinstance(df, pd.DataFrame) and not df.empty:
        try: return float(df["close"].iloc[-1])
        except: return math.nan
    return math.nan

# ---- Export helpers (reliable) ----
def export_csv(df, filename_base="saas_dashboard_data"):
    stamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
    fname = os.path.join(EXPORT_DIR, f"{filename_base}_{stamp}.csv")
    df.to_csv(fname, index=False)
    return fname

def export_png(fig, filename_base="saas_dashboard_image"):
    stamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
    fname = os.path.join(EXPORT_DIR, f"{filename_base}_{stamp}.png")
    fig.savefig(fname, dpi=200, bbox_inches="tight", facecolor=BG)
    return fname

# ---------------- Widgets UI ----------------
if widgets is None:
    print("ipywidgets not installed — running with default selection (first 6 companies).")
    selected_default = COMPANIES[:6]
else:
    checks = [widgets.Checkbox(value=False, description=c) for c in COMPANIES]
    btn_all    = widgets.Button(description="Select All",   button_style="info",    layout=widgets.Layout(width="110px"))
    btn_clear  = widgets.Button(description="Clear All",    button_style="warning", layout=widgets.Layout(width="110px"))
    btn_render = widgets.Button(description="Render Dashboard", button_style="primary", layout=widgets.Layout(width="170px"))
    btn_export = widgets.Button(description="Export CSV/PNG",   button_style="success", layout=widgets.Layout(width="150px"))

    # Forecasting widgets
    company_select = widgets.Dropdown(options=COMPANIES, value=COMPANIES[0], description="Forecast:")
    days_slider = widgets.IntSlider(value=30, min=7, max=90, step=1, description="Horizon (days):", layout=widgets.Layout(width="320px"))
    lstm_epochs = widgets.IntSlider(value=8, min=4, max=50, step=1, description="LSTM epochs:", layout=widgets.Layout(width="320px"))

    control_box = widgets.HBox([btn_all, btn_clear, btn_render, btn_export])
    right_box = widgets.VBox([company_select, days_slider, lstm_epochs])
    grid = widgets.GridBox(children=checks, layout=widgets.Layout(grid_template_columns="repeat(5, 220px)"))
    display(widgets.VBox([widgets.HBox([control_box, right_box]), grid]))
    out = widgets.Output()
    display(out)

    def sel_all(b):
        for cb in checks: cb.value = True
    def sel_clear(b):
        for cb in checks: cb.value = False
    btn_all.on_click(sel_all); btn_clear.on_click(sel_clear)
    for cb in checks[:6]: cb.value = True

_last = {"fig": None, "csv": None, "png": None}

# ---------------- Main polished dashboard builder ----------------
def draw_saas_dashboard(selected_companies, forecast_company=None, forecast_days=30, lstm_epochs_val=8):
    # Step 1: collect data
    trends = simulate_trends(selected_companies, days=90, seed=2025)
    headlines = {}
    sentiment_rows = []
    stocks = {}
    latest_prices = {}

    for c in selected_companies:
        titles = safe_rss_titles(c, max_items=6)
        if not titles:
            titles = [f"{c} posts strong results", f"{c} expands operations", f"{c} partners with startup"]
        headlines[c] = titles
        for t in titles:
            lab, sc = tb_sentiment(t)
            sentiment_rows.append({"company":c, "text":t, "label":lab, "score":sc})
        sym = SYMBOLS.get(c)
        df_stock = fetch_stock_close(sym, days=720) if sym else pd.DataFrame()
        stocks[c] = df_stock
        latest_prices[c] = safe_latest_price(df_stock)

    sent_df = pd.DataFrame(sentiment_rows)
    if sent_df.empty:
        sent_counts = pd.DataFrame(0, index=selected_companies, columns=["Positive","Neutral","Negative"])
    else:
        sent_counts = sent_df.groupby(["company","label"]).size().unstack(fill_value=0)
    for col in ["Positive","Neutral","Negative"]:
        if col not in sent_counts: sent_counts[col] = 0
    sent_counts = sent_counts.reindex(index=selected_companies).fillna(0)

    # Summary table dataframe (clean)
    summary_rows = []
    for c in selected_companies:
        articles = len(headlines.get(c, []))
        avg_sent = round(sent_df[sent_df["company"]==c]["score"].mean() if not sent_df.empty else 0.0, 3)
        price = latest_prices.get(c, math.nan)
        summary_rows.append({"Company":c, "Articles":articles, "Avg_Sent":avg_sent, "Latest_Price": (round(price,2) if not math.isnan(price) else "N/A")})
    summary_df = pd.DataFrame(summary_rows).sort_values("Avg_Sent", ascending=False)

    # Save sentiment CSV for export preview
    stamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
    csv_path = os.path.join(EXPORT_DIR, f"saas_sentiment_{stamp}.csv")
    sent_df.to_csv(csv_path, index=False)

    # Forecasting placeholders (the user can enable; safe errors are caught)
    arima_series, arima_fc = None, None
    lstm_series, lstm_fc = None, None
    arima_msg, lstm_msg = "", ""
    if forecast_company:
        dfc = stocks.get(forecast_company)
        if dfc is None or dfc.empty:
            arima_msg = f"No stock history for {forecast_company}."
            lstm_msg = arima_msg
        else:
            price_series = dfc.set_index("date")["close"].asfreq("D").fillna(method="ffill").dropna()
            arima_series = price_series
            lstm_series = price_series
            try:
                if STATSMODELS_AVAILABLE:
                    arima_fc = arima_forecast(arima_series, steps=forecast_days, order=(5,1,0))
                else:
                    arima_msg = "statsmodels not available."
            except Exception as e:
                arima_msg = f"ARIMA: {e}"
            try:
                if TENSORFLOW_AVAILABLE:
                    lstm_fc = lstm_forecast(lstm_series, steps=forecast_days, lookback=30, epochs=lstm_epochs_val, batch_size=8, verbose=0)
                else:
                    lstm_msg = "TensorFlow not available."
            except Exception as e:
                lstm_msg = f"LSTM: {e}"

    # Step 2: plotting — improved alignment, professional look
    sns.set_style("white")
    plt.rcParams.update({
        "figure.facecolor": BG,
        "axes.facecolor": CARD_BG,
        "savefig.facecolor": BG,
        "font.family": "sans-serif",
        "text.color": HEADER,
        "axes.labelcolor": HEADER,
        "xtick.color": MUTED,
        "ytick.color": MUTED
    })
    fig = plt.figure(constrained_layout=False, figsize=(20,12), facecolor=BG)
    gs = fig.add_gridspec(8, 14, hspace=0.6, wspace=0.6)

    # Header
    ax_header = fig.add_subplot(gs[0, :14]); ax_header.axis("off")
    header_rect = patches.FancyBboxPatch((0,0),1,1, transform=ax_header.transAxes, boxstyle="round,pad=0.02", fc=CARD_BG, ec="#e6eef6")
    ax_header.add_patch(header_rect)
    ax_header.text(0.02, 0.58, "Modern SaaS Market Dashboard — Polished", fontsize=18, fontweight=700, color=ACCENT)
    ax_header.text(0.02, 0.28, "Better alignment, clean tables, and reliable exports", fontsize=10, color=MUTED)
    ax_header.text(0.82, 0.55, f"Updated: {datetime.utcnow().strftime('%Y-%m-%d %H:%M UTC')}", fontsize=9, color=MUTED, ha='right')
    if os.path.exists(LOGO_PATH):
        try:
            logo = Image.open(LOGO_PATH).convert("RGBA")
            ax_logo = fig.add_axes([0.925, 0.02, 0.05, 0.12])
            ax_logo.imshow(logo); ax_logo.axis("off")
        except Exception:
            pass

    # KPI cards row (4 cards)
    kpi_titles = ["Total Mentions", "Avg Sentiment", "Companies", "Top Interest"]
    total_mentions = len(sent_df)
    avg_sent_overall = round(sent_df["score"].mean() if not sent_df.empty else 0.0, 3)
    companies_count = len(selected_companies)
    trend_latest = trends.iloc[-1] if not trends.empty else pd.Series({c:0 for c in selected_companies})
    top_interest_name = trend_latest.idxmax() if not trend_latest.empty else selected_companies[0]
    kpi_values = [total_mentions, avg_sent_overall, companies_count, top_interest_name]

    for i,(title,val) in enumerate(zip(kpi_titles,kpi_values)):
        ax = fig.add_subplot(gs[1, i*3:(i+1)*3]); ax.axis("off")
        card = patches.FancyBboxPatch((0,0),1,1, transform=ax.transAxes, boxstyle="round,pad=0.03,rounding_size=0.05", fc=CARD_BG, ec="#e6eef6")
        ax.add_patch(card)
        ax.text(0.05, 0.62, title, fontsize=10, color=MUTED)
        ax.text(0.05, 0.18, str(val), fontsize=20, fontweight=700, color=ACCENT if i!=2 else HEADER)
        if title == "Avg Sentiment": ax.text(0.95, 0.18, "(-1..+1)", fontsize=8, color=MUTED, ha='right')

    # Main trend (left) and sentiment (right)
    ax_trend = fig.add_subplot(gs[2:4, 0:8])
    colors = sns.color_palette("tab10", n_colors=len(selected_companies))
    for i,c in enumerate(selected_companies):
        ax_trend.plot(trends.index, trends[c], label=c, linewidth=1.4, color=colors[i], alpha=0.9)
    ax_trend.plot(trends.index, trends.mean(axis=1), label="Average", color=ACCENT, linewidth=2.0, linestyle='--', alpha=0.95)
    ax_trend.set_title("Search Interest — Last 90 Days", fontsize=12, color=HEADER)
    ax_trend.set_ylabel("Interest")
    ax_trend.grid(alpha=0.08)
    ax_trend.legend(frameon=False, fontsize=8, loc='upper left')

    ax_sent = fig.add_subplot(gs[2:4, 8:14])
    comp_order = sent_counts.index.tolist()
    y = np.arange(len(comp_order))
    pos_vals = sent_counts["Positive"].values if "Positive" in sent_counts else np.zeros(len(comp_order))
    neu_vals = sent_counts["Neutral"].values if "Neutral" in sent_counts else np.zeros(len(comp_order))
    neg_vals = sent_counts["Negative"].values if "Negative" in sent_counts else np.zeros(len(comp_order))
    ax_sent.barh(y, pos_vals, color=POS, label="Positive")
    ax_sent.barh(y, neu_vals, left=pos_vals, color="#9aa9bf", label="Neutral")
    ax_sent.barh(y, neg_vals, left=pos_vals+neu_vals, color=NEG, label="Negative")
    ax_sent.set_yticks(y); ax_sent.set_yticklabels(comp_order, color=HEADER)
    ax_sent.invert_yaxis()
    ax_sent.set_title("Sentiment Breakdown", fontsize=12, color=HEADER)
    ax_sent.legend(frameon=False, fontsize=8)

    # Stocks mini overview (bottom-left)
    ax_stock = fig.add_subplot(gs[4:6, 0:8])
    for c in selected_companies:
        dfc = stocks.get(c)
        if dfc is not None and not dfc.empty:
            ax_stock.plot(dfc["date"], dfc["close"], linewidth=1.0, alpha=0.8, label=c)
    ax_stock.set_title("Stock Close Overview (~720 days)", fontsize=11, color=HEADER)
    ax_stock.tick_params(axis='x', rotation=18)
    ax_stock.legend(fontsize=7, ncol=3)

    # News panel (middle-bottom)
    ax_news = fig.add_subplot(gs[4:6, 8:11])
    ax_news.axis("off")
    ax_news.set_title("Top News & Social Snippets", fontsize=11, color=HEADER)
    news_lines = []
    for c in selected_companies:
        for t in headlines.get(c, [])[:2]:
            news_lines.append(f"{c}: {t}")
    news_text = "\n\n".join([textwrap.fill(f"{i+1}. {s}", width=50) for i,s in enumerate(news_lines[:18])])
    ax_news.text(0, 0.98, news_text, va='top', color=MUTED, fontsize=9, family='monospace')

    # Summary table (matplotlib.table for alignment & styling)
    ax_table = fig.add_subplot(gs[4:6, 11:14]); ax_table.axis("off")
    ax_table.set_title("Summary — Company Overview", fontsize=11, color=HEADER, pad=6)
    # Use Matplotlib table for proper alignment
    display_df = summary_df.copy().reset_index(drop=True)
    display_df["Latest_Price"] = display_df["Latest_Price"].astype(str)
    table_ax = ax_table
    table_ax.axis("off")
    tb = mtable.table(
        table_ax, cellText=display_df.values, colLabels=display_df.columns,
        colLoc='left', cellLoc='left', loc='center'
    )
    tb.auto_set_font_size(False)
    tb.set_fontsize(9)
    # adjust column widths
    for (row, col), cell in tb.get_celld().items():
        cell.set_edgecolor("#e6eef6")
        if row == 0:
            cell.set_text_props(weight='bold', color=HEADER)
            cell.set_facecolor(TABLE_HEADER_BG)
        else:
            cell.set_facecolor(CARD_BG)
    # shrink the table to fit card with padding
    table_ax.set_ylim(0, 1)
    table_ax.set_xlim(0, 1)

    # Forecast panel (bottom row) — ARIMA/LSTM plots + diagnostics column
    ax_arima = fig.add_subplot(gs[6:8, 0:6])
    ax_lstm  = fig.add_subplot(gs[6:8, 6:12])
    ax_diag  = fig.add_subplot(gs[6:8, 12:14])
    ax_diag.axis("off")

    if arima_series is None:
        ax_arima.text(0.05, 0.5, "No historical prices available for ARIMA/LSTM.\nRender with stocks available to see forecast plots.", fontsize=10, color=MUTED)
        ax_lstm.text(0.05, 0.5, "No historical prices available for ARIMA/LSTM.", fontsize=10, color=MUTED)
    else:
        ax_arima.plot(arima_series.index[-180:], arima_series.values[-180:], label="Actual (last 180d)", color="#4c78a8")
        if arima_fc is not None:
            ax_arima.plot(arima_fc.index, arima_fc.values, label=f"ARIMA Forecast ({forecast_days}d)", color="#ff7f0e", linestyle="--")
        ax_arima.set_title(f"ARIMA Forecast — {forecast_company}", color=HEADER)
        ax_arima.legend(fontsize=8); ax_arima.grid(alpha=0.08)

        ax_lstm.plot(lstm_series.index[-180:], lstm_series.values[-180:], label="Actual (last 180d)", color="#4c78a8")
        if lstm_fc is not None:
            ax_lstm.plot(lstm_fc.index, lstm_fc.values, label=f"LSTM Forecast ({forecast_days}d)", color="#2ca02c", linestyle="--")
        ax_lstm.set_title(f"LSTM Forecast — {forecast_company}", color=HEADER)
        ax_lstm.legend(fontsize=8); ax_lstm.grid(alpha=0.08)

    # diagnostics panel
    diag_lines = [
        f"Forecast target: {forecast_company}",
        f"Horizon (days): {forecast_days}",
        f"ARIMA available: {STATSMODELS_AVAILABLE}",
        f"LSTM available: {TENSORFLOW_AVAILABLE}",
    ]
    if arima_msg: diag_lines.append("ARIMA: " + arima_msg)
    if lstm_msg: diag_lines.append("LSTM: " + lstm_msg)
    ax_diag.text(0.02, 0.98, "\n".join(diag_lines), va="top", fontsize=9, family='monospace', color=MUTED)

    plt.tight_layout(rect=[0, 0.01, 1, 0.97])
    plt.show()

    # Save CSV & PNG and keep paths in _last
    try:
        last_csv = export_csv(sent_df, filename_base="saas_sentiment")
        _last["csv"] = last_csv
    except Exception as e:
        _last["csv"] = None
        print("CSV save failed:", e)
    try:
        last_png = export_png(fig, filename_base="saas_dashboard")
        _last["png"] = last_png
        _last["fig"] = fig
    except Exception as e:
        _last["png"] = None
        print("PNG save failed:", e)

    # final messages
    print("Done — dashboard rendered.")
    if _last["csv"]: print("Saved CSV:", _last["csv"])
    if _last["png"]: print("Saved PNG:", _last["png"])

# --------- button wiring if widgets exist ----------
if widgets is None:
    draw_saas_dashboard(selected_default, forecast_company=COMPANIES[0], forecast_days=30, lstm_epochs_val=8)
else:
    def on_render(b):
        with out:
            clear_output(wait=True)
            sel = [cb.description for cb in checks if cb.value]
            if not sel:
                print("Select at least one company (or use Select All).")
                return
            fc_company = company_select.value
            fc_days = int(days_slider.value)
            fc_epochs = int(lstm_epochs.value)
            print(f"Rendering dashboard for {len(sel)} companies. Forecast: {fc_company} for {fc_days} days (LSTM epochs: {fc_epochs})\n")
            draw_saas_dashboard(sel, forecast_company=fc_company, forecast_days=fc_days, lstm_epochs_val=fc_epochs)

    def on_export(b):
        with out:
            clear_output(wait=True)
            if _last.get("csv"): print("CSV:", _last["csv"])
            else: print("No CSV exported yet. Render the dashboard first.")
            if _last.get("png"): print("PNG:", _last["png"])
            else: print("No PNG exported yet. Render the dashboard first.")

    btn_render.on_click(on_render)
    btn_export.on_click(on_export)

print("Polished SaaS dashboard ready. Logo path (local):", LOGO_PATH)


Output()

Polished SaaS dashboard ready. Logo path (local): /mnt/data/734655bf-327e-4fd5-bcbc-054bb619f15d.png
